## Object Following (50 pts)

In this notebook we'll show how you can follow an object with the crazyflie!  We'll use a pre-trained neural network that was trained on the [COCO dataset](http://cocodataset.org) to detect 90 different common objects.  These include

* Person (index 0)
* Cup (index 47)

and many others (you can check [this file](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt) for a full list of class indices). We use the MobileNet SSD (Single Shot Detector) trained on the COCO dataset. SSD models are often faster than other detection models and the MobileNet backbone is less computationally intensive, so this will help for real-time execution! The model is sourced from the [TensorFlow object detection API](https://github.com/tensorflow/models/tree/master/research/object_detection),
which provides utilities for training object detectors for custom tasks also!

We won't run through all of the training and optimization steps in this notebook though. The goal here is to demonstrate what one can do with neural networks. 

Anyways, let's get started!  First, we will load the pre-trained network. Make sure to have the Lab8_Supplement directory downloaded. Also download the model and place in the Lab8_Supplement directory [https://drive.google.com/file/d/1vIS9XySf5kdmVqPCtCpHG_-FL6RB8oOP/view](https://drive.google.com/file/d/1vIS9XySf5kdmVqPCtCpHG_-FL6RB8oOP/view).

### Compute detections on single camera image

For this lab, we will be using OpenCV's DNN module which provides us with functionalities for deep learning inference. You can read more about how we are using it for [object detection](https://learnopencv.com/deep-learning-with-opencvs-dnn-module-a-definitive-guide/). Specifically, we can load in the MobileNet SSD network that was trained using the Tensorflow framework. OpenCV's DNN module allows for multi-framework use (e.g., PyTorch and Caffe).

First, we load in the COCO class names (e.g., person, potted plant, etc.), assign colors to the classes (this is useful for visualizing bounding boxes), and load the weights of the pre-trained neural network. 

In [1]:
import cv2
import numpy as np

# load the COCO class names
with open('Lab8_Supplement/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')
    
# get a different color array for each of the classes
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

# load the DNN model
model = cv2.dnn.readNet(model='Lab8_Supplement/frozen_inference_graph.pb',
                        config='Lab8_Supplement/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt', 
                        framework='TensorFlow')

Now we will prepare an image for object detection with our model. `blobFromImage()` prepares the image into the correct format for our model. Specifically, we resize our input image to 300x300 and normalize the RGB channels with the mean parameter. Then we forward propagate the image through the model to obtain the detections. Each detection is of the form ( _, class_id, confidence, box_x, box_y, box_width, box_height) where box_x, box_y, box_width, box_height provide information for creating the bounding box of around the detected object.

In [2]:
# read the image from disk
image = cv2.imread('Lab8_Supplement/Lab8_image.jpg')
image_height, image_width, _ = image.shape

# create blob from image
blob = cv2.dnn.blobFromImage(image=image, size=(300, 300), mean=(104, 117, 123), 
                             swapRB=True)

# create blob from image
model.setInput(blob)

# forward pass through the model to carry out the detection
detections = model.forward()

Next we visualize the detections. You should see a bounding box, classification, and confidence value appear around each COCO object (potted plant and cup).

In [3]:
# loop over each of the detection
for detection in detections[0, 0, :, :]:
    # extract the confidence of the detection
    confidence = detection[2]
    # draw bounding boxes only if the detection confidence is above...
    # ... a certain threshold, else skip
    if confidence > .4:
        # get the class id
        class_id = detection[1]
        # map the class id to the class
        class_name = class_names[int(class_id)-1]
        color = COLORS[int(class_id)]
        # get the bounding box coordinates
        box_x = detection[3] * image_width
        box_y = detection[4] * image_height
        # get the bounding box width and height
        box_width = detection[5] * image_width
        box_height = detection[6] * image_height
        # draw a rectangle around each detected object
        cv2.rectangle(image, (int(box_x), int(box_y)), (int(box_width), int(box_height)), color, thickness=1)
        # put the FPS text on top of the frame
        text = class_name + ' ' + '%.2f' % (confidence)
        cv2.putText(image, text, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, color, 1)

while(True):
    cv2.imshow('image', image)
    
    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cv2.imwrite('Lab8_Supplement/image_result.jpg', image)


True

To print just the first object detected in the example image, we could call the following:

In [4]:
object_number = 0
print(detections[0, 0, object_number, :])

[ 0.         64.          0.8862912   0.3806074   0.31562245  0.61905956
  0.71215916]


### Compute detections on a live video feed

The following cell will perform the same object detection and labeling on a live feed from your CrazyFlie camera! Note that the drone will not fly, you are simply using the camera. This should give you a sense of appropriate distances for detection, as well as the confidence for the detection of different objects from the Coco dataset.

In [5]:
# This may open your webcam instead of the CrazyFlie camera! If so, try
# a different small, positive integer, e.g. 1, 2, 3.
camera = 0
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    image_height, image_width, _ = frame.shape

    # create blob from image
    blob = cv2.dnn.blobFromImage(image=frame, size=(300, 300), mean=(104, 117, 123), 
                                 swapRB=True)

    # create blob from image
    model.setInput(blob)

    # forward pass through the model to carry out the detection
    detections = model.forward()

    # loop over each of the detection
    for detection in detections[0, 0, :, :]:
        # extract the confidence of the detection
        confidence = detection[2]
        # draw bounding boxes only if the detection confidence is above...
        # ... a certain threshold, else skip
        if confidence > .4:
            # get the class id
            class_id = detection[1]
            # map the class id to the class
            class_name = class_names[int(class_id)-1]
            color = COLORS[int(class_id)]
            # get the bounding box coordinates
            box_x = detection[3] * image_width
            box_y = detection[4] * image_height
            # get the bounding box width and height
            box_width = detection[5] * image_width
            box_height = detection[6] * image_height
            # draw a rectangle around each detected object
            cv2.rectangle(frame, (int(box_x), int(box_y)), (int(box_width), int(box_height)), color, thickness=1)
            # put the FPS text on top of the frame
            text = class_name + ' ' + '%.2f' % (confidence)
            cv2.putText(frame, text, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, color, 1)
    
    # Compute
    cv2.imshow('frame', frame)    

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

[ WARN:0@4.646] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


I have built some test code to test proximity threshhold before running the code. 

In [6]:
# # This may open your webcam instead of the CrazyFlie camera! If so, try
# # a different small, positive integer, e.g. 1, 2, 3.
# camera = 0
# cap = cv2.VideoCapture(camera)

# # Proximity threshold for determining "closeness" (this could be adjusted based on your testing)
# proximity_threshold = 500  # You can adjust this value based on the bounding box size when close to the object

# while(True):
#     # Capture frame-by-frame
#     ret, frame = cap.read()

#     image_height, image_width, _ = frame.shape

#     # create blob from image
#     blob = cv2.dnn.blobFromImage(image=frame, size=(300, 300), mean=(104, 117, 123), 
#                                  swapRB=True)

#     # create blob from image
#     model.setInput(blob)

#     # forward pass through the model to carry out the detection
#     detections = model.forward()

#     ########################
#     # Variable to track if a close object is found
#     ########################
#     close_to_object = False


#     # loop over each of the detection
#     for detection in detections[0, 0, :, :]:
#         # extract the confidence of the detection
#         confidence = detection[2]
#         # draw bounding boxes only if the detection confidence is above...
#         # ... a certain threshold, else skip
#         if confidence > .4:
#             # get the class id
#             class_id = detection[1]
#             # map the class id to the class
#             class_name = class_names[int(class_id)-1]
#             color = COLORS[int(class_id)]
#             # get the bounding box coordinates
#             box_x = detection[3] * image_width
#             box_y = detection[4] * image_height
#             # get the bounding box width and height
#             box_width = detection[5] * image_width
#             box_height = detection[6] * image_height
#             # draw a rectangle around each detected object
#             cv2.rectangle(frame, (int(box_x), int(box_y)), (int(box_width), int(box_height)), color, thickness=1)
#             # put the FPS text on top of the frame
#             text = class_name + ' ' + '%.2f' % (confidence)
#             cv2.putText(frame, text, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, color, 1)
            
#             ################
#             # Proximity check: if the bounding box is large enough, consider it "close"
#             ################
#             if box_width > proximity_threshold or box_height > proximity_threshold:
#                 close_to_object = True


#     # Compute
#     cv2.imshow('frame', frame)    

#     # Print message if close to an object
#     if close_to_object:
#         print("Object is close to the camera!")
#     else:
#         print("not close to camera")

#     # Hit q to quit.
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Release the capture
# cap.release()
# cv2.destroyAllWindows()

In [7]:
print(image_height)
print(image_width)

480
640


### Control robot to follow central object

Now we want our robot to follow an object of the specified category (e.g., person, etc.).  To do this we'll do the following

1.  Detect objects matching the specified class
2.  Select object closest to center of camera's field of vision; this is the 'target' object
3.  Control the robot towards target object; otherwise hover

We'll also create a controller that will use the distance between the target object and the center of the robot's field of view to follow the object as well as use the bounding box size to determine when to stop. 

First, let's define some functions that will process the images from the crazyflie. 

### Task 1 (10 pts) ###

Fill in the function "closest_detection" below. This should find the detected object that is closest to the center of the image. 

In [8]:
import time

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    center_x = (detection[3] + detection[5]) / 2.0 - 0.5
    center_y = (detection[4] + detection[6]) / 2.0 - 0.5
    return (center_x, center_y)

def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """TODO: Find the detection closest to the image center"""
    # Loop through and find the detection that is closest to the image center
    # You can use the detection_center function above to find the center of the detected object
    # Note that the origin (i.e., (x,y) = (0,0)) corresponds to the center of the image. So you can
    # use the "norm" function above to find the detection that is closest to the center.
    # Return the det that corresponds to the closest detection to the image center.
    # If nothing is detected, return None.

    if not detections:
        return None

    closest_det = None
    min_distance = float('inf')

    for det in detections: # Loop through and find the detection closes to center
        center = detection_center(det) #Use detection center
        distance = norm(center) #Compute distances 

        if distance < min_distance:
            min_distance = distance
            closest_det = det
    return closest_det

Great, now let's get ready to control the crazyflie to follow an object! Below are a few functions to help move the crazyflie.

In [9]:
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander
from cflib.positioning.motion_commander import MotionCommander


def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001
    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            print("{} {} {}".
                format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break

def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    
    wait_for_position_estimator(cf)
    time.sleep(0.1)    
    return

# Ascend and hover:
def ascend_and_hover(cf):
    # Ascend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 10)
        time.sleep(0.1)
    # Hover at 1 meter:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    return

def hover(cf):
    print('Hovering:')
    # Hover at 1 meter:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    return
    
# Hover, descend, and stop all motion:
def hover_and_descend(cf):
    # Hover at 1 meter:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 1)
        time.sleep(0.1)
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 10)
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return

### Task 2 (20 pts) ###

Fill in the controller below that says "TODO" to make the crazyflie follow the object. The controller should use the inputs to keep the detected target in the center of its view as well determine when to stop (send True flag) so that the crazyflie stops and lands before crashing into the tracked object. (Note: the execution code implements the actual stopping using the flag).

In [10]:
# def controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur):
#     """
    
#     cf: crazyflie instance
#     box_x: x coordinate of the center of the bounding box in the image
#     box_y: y coordinate of the center of the bounding box in the image
#     box_width: width of the bounding box in the image
#     box_height: height of the bounding box in the image
#     x_cur: current x position
#     y_cur: current y position
    
#     Return True to indicate that the drone is close to the target and thus exit the loop to stop and descend, new x, new y
#     Return False to indicate continuing to follow the target, new x, new y.
    
#     """
    
#     #### TO DO: Fill below ####
#     # Exit condition/method using size of the bounding box
#     proximity_threshold = 300 #150 #tbd
#     if box_width > proximity_threshold or box_height > proximity_threshold:
#         return True, x_cur, y_cur
    
#     #### TO DO: Fill below ####
#     # Determine the x and y velocity
#     x_adjustment = -0.01 * box_x
#     y_adjustment = -0.01 * box_y
#     x_command = x_cur + x_adjustment
#     y_command = y_cur + y_adjustment



#     # Set velocity
#     cf.commander.send_position_setpoint(x_command, y_command, 1, 0) # Do not edit this line

#     #Return false if you're not close to the target, return true if you are
#     return False, x_command, y_command
    
    

In [11]:
# def controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur):
#     """
#     Controller for moving the Crazyflie drone based on the object's position.

#     cf: Crazyflie instance
#     box_x: x-coordinate of the center of the bounding box in the image
#     box_y: y-coordinate of the center of the bounding box in the image
#     box_width: width of the bounding box in the image
#     box_height: height of the bounding box in the image
#     x_cur: current x position
#     y_cur: current y position

#     Return True to indicate that the drone is close to the target and should stop/descend,
#     or False to continue tracking the object, and the new x, y position.
#     """
    
#     # Define proximity threshold: when the bounding box is large enough to stop (object is close)
#     proximity_threshold = 500  # Can be adjusted based on your testing
    
#     # If bounding box is large enough, assume we are close to the target and want to stop/descend
#     if box_width > proximity_threshold or box_height > proximity_threshold:
#         print("OMG Target is close enough. Preparing to stop or descend.")
#         return True, x_cur, y_cur  # Return True to exit tracking, descend, or stop
    
#     # # If no detection, we keep the drone hovering
#     # if box_width == 0 or box_height == 0:
#     #     print("Bruh No detection...hovering.")
#     #     cf.commander.send_position_setpoint(x_cur, y_cur, 1, 0)
#     #     return False, x_cur, y_cur  # Continue hovering at the same position
    
#     # Calculate the x and y adjustments to move towards the target
#     # The further the box center is from the center of the image, the stronger the correction will be
#     x_adjustment = -0.01 * box_x  # Adjust based on the center x position
#     y_adjustment = -0.01 * box_y  # Adjust based on the center y position

#     # Compute new target positions
#     x_command = x_cur + x_adjustment
#     y_command = y_cur + y_adjustment

#     # Send position setpoint command to Crazyflie
#     cf.commander.send_position_setpoint(x_command, y_command, 1, 0)  # Z is set to 1 (e.g., 1m above the ground)

#     # Return false to continue following the target, and update current positions
#     return False, x_command, y_command


In [12]:
# def controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur):
#     """
#     cf: crazyflie instance
#     box_x: x coordinate of the center of the bounding box in the image
#     box_y: y coordinate of the center of the bounding box in the image
#     box_width: width of the bounding box in the image
#     box_height: height of the bounding box in the image
#     x_cur: current x position
#     y_cur: current y position
    
#     Return True to indicate that the drone is close to the target and thus exit the loop to stop and descend, new x, new y
#     Return False to indicate continuing to follow the target, new x, new y.
#     """

#     # Calculate the size of the bounding box (which is used as an indication of distance)
#     box_size = box_width * box_height  # Area of the bounding box



#     # Define constants for the exit condition
#     EXIT_BOX_SIZE = 50  # Threshold bounding box size to trigger stop (adjust as needed)
#     CENTER_X = #320  # Assuming the image width is 640px, center is at 320px
#     CENTER_Y = 240  # Assuming the image height is 480px, center is at 240px
    
#     # Calculate the error in position (offset from the center of the image)
#     error_x = box_x - CENTER_X  # Horizontal error
#     error_y = box_y - CENTER_Y  # Vertical error
    
#     # If the bounding box size is greater than the threshold, stop the drone
#     if box_size > EXIT_BOX_SIZE:
#         # If the target is big enough, we are close, so stop the drone and return True
#         cf.commander.send_position_setpoint(x_cur, y_cur, 1, 0)  # Stop command (stay in position)
#         return True, x_cur, y_cur
    
#     # Determine velocity: we can use a simple proportional controller    
#     # Calculate the desired position adjustments based on the error
#     x_command = x_cur + (-0.0001 * error_x)  # Adjust in x direction
#     y_command = y_cur + (-0.0001 * error_y)  # Adjust in y direction

#     # Send the new position setpoint to Crazyflie
#     cf.commander.send_position_setpoint(x_command, y_command, 1, 0)  # Set position to move towards

#     # Return False to continue the loop, with the new command positions
#     return False, x_command, y_command


In [13]:
# def controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur):
#     """
#     EXPERIMENT
#     """
    
#     # Set proximity threshold based on object size (experimentally determined)
#     proximity_threshold = 75  # This may need adjustment depending on the object’s distance

#     # Condition to stop and land if the bounding box is close enough to the threshold size
#     if box_width > proximity_threshold or box_height > proximity_threshold:
#         return True, x_cur, y_cur  # Stop following and land

#     # Condition to handle lost object case (when bounding box disappears or is too small)
#     # if box_width <= 5 or box_height <= 5:
#     #     # If the object cannot be detected reliably, stop movement to avoid risk
#     #     return False, x_cur, y_cur
#         #return True, x_cur, y_cur  # Stop and land if the object is lost

#     # Calculate the adjustment needed to keep the object centered in the view
#     # Adjustments scaled down for stability
#     adjustment_factor = 0.001
#     x_adjustment = -adjustment_factor * box_x
#     y_adjustment = -adjustment_factor * box_y

#     # New target positions based on current position
#     # Limits are set to avoid abrupt movements
#     x_command = x_cur + max(min(x_adjustment, 0.2), -0.2)
#     y_command = y_cur + max(min(y_adjustment, 0.2), -0.2)

#     # Command the Crazyflie to move to the calculated position
#     cf.commander.send_position_setpoint(x_command, y_command, 1, 0)

#     # Continue following if not yet close to the object
#     return False, x_command, y_command


In [14]:
# def controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur):
#     """
#     EXPERIMENT
#     """
    
#     # Set proximity threshold based on object size (experimentally determined)
#     proximity_threshold = 100  # This may need adjustment depending on the object’s distance

#     # Condition to stop and land if the bounding box is close enough to the threshold size
#     if box_width > proximity_threshold or box_height > proximity_threshold:
#         return True, x_cur, y_cur  # Stop following and land

#     # Condition to handle lost object case (when bounding box disappears or is too small)
#     if box_width <= 5 or box_height <= 5:
#         # If the object cannot be detected reliably, hover in place
#         cf.commander.send_position_setpoint(x_cur, y_cur, 1, 0)
#         return False, x_cur, y_cur  # Continue hovering

#     # Calculate the adjustment needed to keep the object centered in the view
#     # Adjustments scaled down for stability
#     adjustment_factor = 0.001
#     x_adjustment = -adjustment_factor * box_x
#     y_adjustment = -adjustment_factor * box_y

#     # New target positions based on current position
#     # Limits are set to avoid abrupt movements
#     x_command = x_cur + max(min(x_adjustment, 0.2), -0.2)
#     y_command = y_cur + max(min(y_adjustment, 0.2), -0.2)

#     # Command the Crazyflie to move to the calculated position
#     cf.commander.send_position_setpoint(x_command, y_command, 1, 0)

#     # Continue following if not yet close to the object
#     return False, x_command, y_command


In [15]:
# def controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur):
#     """
#     Makes the drone move towards the detected object until proximity threshold is met.
#     """
    
#     # Set proximity threshold based on object size (experimentally determined)
#     proximity_threshold = 75  # Adjust this value based on object distance and size

#     # Condition to check if the object is close enough to the threshold size
#     if box_width > proximity_threshold or box_height > proximity_threshold:
#         return True, x_cur, y_cur  # Stop moving; proximity threshold reached

#     # Condition to handle lost object case (when bounding box disappears or is too small)
#     if box_width <= 5 or box_height <= 5:
#         # If the object cannot be detected reliably, hover in place
#         cf.commander.send_position_setpoint(x_cur, y_cur, 1, 0)
#         return False, x_cur, y_cur  # Continue hovering

#     # Calculate the adjustment needed to move towards the object
#     # The object's bounding box center determines the direction
#     adjustment_factor = 0.001  # Scaling factor for smooth movements
#     x_adjustment = -adjustment_factor * box_x  # Negative because we want to move towards center
#     y_adjustment = -adjustment_factor * box_y

#     # New target positions based on current position
#     # Apply limits to adjustments to avoid sudden movements
#     x_command = x_cur + max(min(x_adjustment, 0.2), -0.2)
#     y_command = y_cur + max(min(y_adjustment, 0.2), -0.2)

#     # Command the Crazyflie to move towards the calculated position
#     cf.commander.send_position_setpoint(x_command, y_command, 1, 0)

#     # Continue following the object
#     return False, x_command, y_command


In [16]:
# def controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur):
#     """
#     cf: crazyflie instance
#     box_x: x coordinate of the center of the bounding box in the image
#     box_y: y coordinate of the center of the bounding box in the image
#     box_width: width of the bounding box in the image
#     box_height: height of the bounding box in the image
#     x_cur: current x position
#     y_cur: current y position
    
#     Return True to indicate that the drone is close to the target and thus exit the loop to stop and descend, new x, new y
#     Return False to indicate continuing to follow the target, new x, new y.
#     """

#     # Calculate the size of the bounding box (which is used as an indication of distance)
#     #bounding_box_x_of_person = box_x

#     current_box_size = box_width * box_height  # Area of the bounding box
#     threshhold_box_size = 400 #291840 #245760 #40000

#     # Define constants for the exit condition
#     #center_x = image_width/2  # If im w 640px, center @ 320px
#     #center_y = image_height/2  # If im h 480px, center @ 240px
#     center_left_to_right = image_height/2 #image_width/2
#     #center_
#     total_image_size = image_width*image_height #307200
    
#     # Calculate the error in position (offset from the center of the image)
#     #error_x = box_x - center_x  # Horizontal error  - once you are aligned, the center of box stays same so you should in fact do this based on size of box
#     error_x = 2 #NEED TO REIMPLEMENT SIZE 
#     error_y = box_y - center_left_to_right  # Vertical error
    
#     # If the bounding box size is greater than the threshold, stop the drone
#     if current_box_size > threshhold_box_size:
#         # If the target is big enough, we are close, so stop the drone and return True
#         cf.commander.send_position_setpoint(x_cur, y_cur, 1, 0)  # Stop command (stay in position)
#         return True, x_cur, y_cur
    
#     # Determine velocity: we can use a simple proportional controller    
#     # Calculate the desired position adjustments based on the error
#     x_command = x_cur + 0*(-0.0001 * error_x)  # Adjust in x direction (this is forward on drone facing me)
#     y_command = y_cur + (0.0001 * error_y)  # Adjust in y direction (this is left and right on drone facing me) - this is good to go

#     # Send the new position setpoint to Crazyflie
#     cf.commander.send_position_setpoint(x_command, y_command, 1, 0)  # Set position to move towards

#     # Return False to continue the loop, with the new command positions
#     return False, x_command, y_command


In [17]:
# def controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur):
#     """
#     cf: crazyflie instance
#     box_x: x coordinate of the center of the bounding box in the image
#     box_y: y coordinate of the center of the bounding box in the image
#     box_width: width of the bounding box in the image
#     box_height: height of the bounding box in the image
#     x_cur: current x position
#     y_cur: current y position
    
#     Return True to indicate that the drone is close to the target and thus exit the loop to stop and descend, new x, new y
#     Return False to indicate continuing to follow the target, new x, new y.
#     """

#     current_box_size = box_width * box_height  # Area of the bounding box
#     total_image_size = image_width*image_height #307200
#     threshhold_box_w = 400 #330 #291840 #245760 #40000
#     threshhold_box_h = 600

#     # Define constants for the exit condition
#     center_left_to_right = image_width/2 #image_height/2 #image_width/2
    
#     # Calculate the error in position (offset from the center of the image)
#     error_x = current_box_size-total_image_size
#     error_y = box_y - center_left_to_right  # Vertical error
    
#     # If the bounding box size is greater than the threshold, stop the drone
#     if box_width > threshhold_box_w or box_height > threshhold_box_h: ##if current_box_size > threshhold_box_size:
#         cf.commander.send_position_setpoint(x_cur, y_cur, 1, 0)  # Stop command (stay in position)
#         return True, x_cur, y_cur  # If the target is big enough, we are close, so stop the drone and return True

#     # Calculate the desired position adjustments based on the error (test by applying 0 on each command)    
#     x_command = x_cur + 0*(-0.0001 * error_x)  # Adjust in x direction (this is forward on drone facing me)
#     y_command = y_cur + -(0.0001 * error_y)  # Adjust in y direction (this is left and right on drone facing me) - this is good to go

#     # Send the new position setpoint to Crazyflie
#     cf.commander.send_position_setpoint(x_command, y_command, 1, 0)  # Set position to move towards

#     # Return False to continue the loop, with the new command positions
#     return False, x_command, y_command


In [18]:
# def controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur):
#     """
#     Controller logic for forward/backward motion based on human stepping forward/backward.
    
#     Parameters:
#     cf: crazyflie instance
#     box_x: x coordinate of the center of the bounding box in the image
#     box_y: y coordinate of the center of the bounding box in the image
#     box_width: width of the bounding box in the image
#     box_height: height of the bounding box in the image
#     x_cur: current x position
#     y_cur: current y position
    
#     Returns:
#     - True to indicate that the drone should stop and descend.
#     - False to indicate continuing to follow the target.
#     - Updated x, y coordinates (new_x, new_y).
#     """
#     # Define the reference bounding box width for "neutral" distance
#     reference_box_width = 200  # Example reference width (tune as necessary)
#     tolerance = 50  # Acceptable error margin around the reference width

#     # Calculate the error between current box width and the reference
#     width_error = box_width - reference_box_width

#     # Define a proportional gain for controlling the movement
#     k_p = 0.01  # Tune this value as needed for smooth response

#     # Calculate new x position based on the error
#     x_command = x_cur - (k_p * width_error)  # Move forward or backward based on error
#     y_command = y_cur  # No change in y for now

#     cf.commander.send_position_setpoint(x_command, y_command, 1, 0)  # Set position to move towards


#     # Check if the drone is within the tolerance range of the target
#     if abs(width_error) < tolerance:
#         return True, x_command, y_command  # Stop the drone when close enough
#     else:
#         return False, x_command, y_command  # Continue adjusting position


In [23]:
def controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur):
    """
    cf: crazyflie instance
    box_x: x coordinate of the center of the bounding box in the image
    box_y: y coordinate of the center of the bounding box in the image
    box_width: width of the bounding box in the image
    box_height: height of the bounding box in the image
    x_cur: current x position
    y_cur: current y position
    
    Return True to indicate that the drone is close to the target and thus exit the loop to stop and descend, new x, new y
    Return False to indicate continuing to follow the target, new x, new y.
    """

    x_center_of_image = image_width/2
    #y_center_of_image = image_height/2

    current_box_size = box_width * box_height  # Area of the bounding box
    total_image_size = image_width*image_height # Area of the image
    
    error_for_horizontal_movement = x_center_of_image-box_x
    error_for_forwards_and_backwards_movement = current_box_size/total_image_size

    
    # If the bounding box to total image ratio is greater than the threshold, stop the drone
    #print(error_for_forwards_and_backwards_movement)
    if error_for_forwards_and_backwards_movement > 1.8*10**-6: #0.6: 
        print("DROP DROP DROP A FIFTY BAG FOR THE MOB IN THE SPOT")
        cf.commander.send_position_setpoint(x_cur, y_cur, 1, 0)  # Stop command (stay in position)
        return True, x_cur, y_cur  # If the target is big enough, we are close, so stop the drone and return True

    # Calculate the desired position adjustments based on the error
    elif error_for_forwards_and_backwards_movement < 1.2*10**-6:
        print("Suspected MOVE FORWARD")
        x_command = x_cur + (10000 * error_for_forwards_and_backwards_movement)  # Adjust in x direction (this is forward on drone facing me)
    elif error_for_forwards_and_backwards_movement > 1.65*10**-6:
        print("Suspected MOVE BACKWARD")
        x_command = x_cur + (-10000 * error_for_forwards_and_backwards_movement)
    else:
        x_command = x_cur

    # x_command = x_cur
    
    #threshhold_for_horizontal_movement = 400 #500
    #print(error_for_horizontal_movement)
    # if error_for_horizontal_movement > threshhold_for_horizontal_movement:
    #     y_command = y_cur + -(0.0000001 * error_for_horizontal_movement)  # Adjust in y direction (this is left and right on drone facing me) - this is good to go
    # elif error_for_horizontal_movement < -threshhold_for_horizontal_movement:
    #     y_command = y_cur + -(0.0000001 * error_for_horizontal_movement)
    # else: 
    #     y_command = y_cur

    y_command = y_cur

    # Send the new position setpoint to Crazyflie
    cf.commander.send_position_setpoint(x_command, y_command, 1, 0)  # Set position to move towards

    # Return False to continue the loop, with the new command positions
    return False, x_command, y_command


The following code will test your controller on the crazyflie. There are several parameters at the top that may be useful to change as indicated, otherwise do not modify the code. Please read the safety and submission instructions below before running.

In [24]:
import cv2
import time
import numpy as np

# load the COCO class names
with open('Lab8_Supplement/object_detection_classes_coco.txt', 'r') as f:
    class_names = f.read().split('\n')

# get a different color array for each of the classes
COLORS = np.random.uniform(0, 255, size=(len(class_names), 3))

# load the DNN model
model = cv2.dnn.readNet(model='Lab8_Supplement/frozen_inference_graph.pb',
                        config='Lab8_Supplement/ssd_mobilenet_v2_coco_2018_03_29.pbtxt.txt', 
                        framework='TensorFlow')

# ************ Parameters that might be useful to change ************ 
# COCO label id that we want to track
tracking_label = 1 # PERSON (1), CHAIR (62)

# Set the URI the Crazyflie will connect to
group_number = 1
uri = f'radio://0/{group_number}/2M'

# Possibly try 0, 1, 2 ...
camera = 0

# Confidence of detection
confidence = 0.4 #0.4

# ******************************************************************

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascend to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        # capture the video
        cap = cv2.VideoCapture(camera)
        
        # get the video frames' width and height
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        # flag indicating whether to exit the main loop and then descend
        exit_loop = False

        # Ascend and hover a bit
        set_PID_controller(cf)
        ascend_and_hover(cf)
        time.sleep(1)
        
        x_cur = 0
        y_cur = 0
        
        # detect objects in each frame of the video
        while cap.isOpened() and not exit_loop:
            
            # Try to read image
            ret, frame = cap.read()
            if ret:
                image = frame
                image_height, image_width, _ = image.shape

                # create blob from image
                blob = cv2.dnn.blobFromImage(image=image, size=(300, 300), mean=(104, 117, 123), 
                                             swapRB=True)

                # forward propagate image
                model.setInput(blob)
                detections = model.forward()

                # select detections that match selected class label
                matching_detections = [d for d in detections[0, 0] if d[1] == tracking_label]

                # select confident detections
                confident_detections = [d for d in matching_detections if d[2] > confidence]

                # get detection closest to center of field of view and draw it
                det = closest_detection(confident_detections) # This relies on the function you wrote above
                
                if det is not None:
                    # get the class id
                    class_id = det[1]
                    # map the class id to the class 
                    class_name = class_names[int(class_id)-1]
                    color = COLORS[int(class_id)]
                    # get the bounding box coordinates
                    box_x = det[3] * image_width
                    box_y = det[4] * image_height
                    # get the bounding box width and height
                    box_width = det[5] * image_width
                    box_height = det[6] * image_height
                    # draw a rectangle around each detected object
                    cv2.rectangle(image, (int(box_x), int(box_y)), (int(box_width), int(box_height)), color, thickness=2)
                    # put the class name text on the detected object
                    cv2.putText(image, class_name, (int(box_x), int(box_y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

                # If nothing is detected, hover
                if det is None:
                    print('no detection...hovering')
                    hover(cf)

                # otherwise  move towards target
                else:
                    print('detection...tracking')
                    _, _, _, box_x, box_y, box_width, box_height = det
                    box_x, box_y = detection_center(det)
                    exit_loop, x_cur, y_cur = controller(cf, box_x, box_y, box_width, box_height, x_cur, y_cur)

                # Check image
                cv2.imshow('image', image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
            else:
                print('no image!!')
                
        cap.release()
        
        # Descend and stop all motion:
        hover_and_descend(cf)
        
    
        cv2.destroyAllWindows()

Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/1/2M
radio://0/1/2M
radio://0/1/2M


[ WARN:0@258.600] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Initializing PID Controller
Waiting for estimator to find position...
999.999987875246 999.9999878782173 999.9997381219291
999.999987875246 999.9999878782173 999.9997434191464
999.999987875246 999.9999878782173 999.9997434191464
999.9999888341135 999.9999887703634 999.9997601928335
999.9999889602577 999.9999889597602 999.9997606851393
999.9999894583871 999.9999894654411 999.9997649408033
999.9999894620187 999.9999894675148 999.9997652086313
999.9999894620187 999.9999894675148 999.9997652086313
999.9999894620187 999.9999894675148 999.9997683609108
1.8804912542691454e-06 1.8876125977840275e-06 3.0238981707952917e-05
detection...tracking
Suspected MOVE FORWARD
detection...tracking
Suspected MOVE FORWARD
detection...tracking
Suspected MOVE FORWARD
detection...tracking
Suspected MOVE FORWARD
detection...tracking
Suspected MOVE FORWARD
detection...tracking
Suspected MOVE FORWARD
detection...tracking
Suspected MOVE FORWARD
detection...tracking
Suspected MOVE FORWARD
detection...tracking
Suspe

<!-- If the previous cell has an error or you lose connection with your drone, run the following cell and restart the kernel. -->

In [ ]:
cap.release()

<!-- # Submission #

Please submit to Gradescope "HW8: Coding" a zip including: this notebook Lab8 (30pts), a video (20pts see below), and Lab9 notebook (50pts).

For the video, please submit the following:
- (10 pts) A video (e.g., taken from your cellphone) showing the crazyflie following you (or any other person). The person should be moving such that it is clear the crazyflie is changing its tracking to follow the person. Read safety instructions below before trying! The crazyflie should stop and land when close to the person. -->

<!-- # Safety #

As always, please wear your safety glasses when working with the crazyflie. 

Additionally, for human tracking, please stand OUTSIDE of the netted test space. The drone's camera is capable of detecting people standing behind the net.  -->